### Создание DataSet

Загрузим все необходимые библиотеки и начнем:

In [1]:
import cv2
import numpy as np
import pickle
import mediapipe as mp

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

Создадим функцию для сериализации видео с помощью модуля pickle

In [3]:
def save_data(obj, name):
    with open('data/' + name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

Создадим датасеты на каждый наш жест 

In [6]:
#Создадим список с нашими жестами
files = ['I.mp4', 'Ok.mp4', 'Palm.mp4', 'Thumb.mp4']
for file in files:
    
    cap = cv2.VideoCapture(file) 
    # Сделаем разгранечения на каждый жест
    label = None
    if file[:1] == 'I':
        label = 0
    if file[:2] == 'Ok':
        label = 1
    if file[:4] == 'Palm':
        label = 2
    if file[:5] == 'Thumb':
        label = 3

    # За основу взят пример распознования рук библиотеки mediapipe 
    data_list=[]
    count=0
    with mp_hands.Hands(
        min_detection_confidence=0.75,
        min_tracking_confidence=0.5,
        max_num_hands=1
    ) as hands:
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring empty camera frame.")
                # If loading a video, use 'break' instead of 'continue'.
                break

            palm_vector_list = []

            # Flip the image horizontally for a later selfie-view display, and convert
            # the BGR image to RGB.
            count=count+1
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            
            # To improve performance, optionally mark the image as not writeable to
            # pass by reference.
            image.flags.writeable = False
            results = hands.process(image)

            # Draw the face mesh annotations on the image.
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    for n_point in range(len(results.multi_hand_landmarks[0].landmark)):
                            palm_vector_list.append(hand_landmarks.landmark[n_point].x)
                            palm_vector_list.append(hand_landmarks.landmark[n_point].y)                        
                            palm_vector_list.append(hand_landmarks.landmark[n_point].z)                
                    mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS) 

                # Добавляем метку каждому жесту жеста
                palm_vector_list.append(label)
                data_list.append(np.array(palm_vector_list))
   
                cv2.putText(image, 
                        'Hands', (100, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                cv2.namedWindow('MediaPipe Hands', cv2.WINDOW_NORMAL) # Были проблемы с отображением (окно больших размеров)
                cv2.imshow('MediaPipe Hands', image)

            if cv2.waitKey(5) & 0xFF == 27:
                break

    d=np.array(data_list)
    save_data(d, 'data__' + str((label)))   
    cap.release()
    cv2.destroyAllWindows()

Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
Ignoring empty camera frame.
